## Motivation

In this note, we exame the idea declared in section 1.5.5 on MNIST data.

In [1]:
import numpy as np
import tensorflow as tf
from keras.losses import MSE
from sklearn.metrics import accuracy_score

from utils import get_gradient_loss_fn

tf.random.set_seed(42)

2024-03-19 16:10:29.029594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## The MNIST Data

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32').reshape([-1, 28*28]) / 255.0
x_test = x_test.astype('float32').reshape([-1, 28*28]) / 255.0
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

## Train a Model with Gradient Loss

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, 'softmax')
])

get_gradient_loss = get_gradient_loss_fn(
    lambda inputs: MSE(inputs[1], model(inputs[0]))
)

In [4]:
optimizer = tf.optimizers.Adam()

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        loss = get_gradient_loss((x, y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables))
    return loss

In [24]:
def evaluate(model):
    return accuracy_score(y_test, tf.argmax(model(x_test), axis=1))

In [5]:
ds = tf.data.Dataset.from_tensor_slices(
    (x_train, tf.one_hot(y_train, 10))
)
ds = ds.batch(100)

In [6]:
for epoch in range(20):
    for x, y in ds:
        loss = train_step(x, y)
    print(epoch, loss)

0 tf.Tensor(0.0002928416, shape=(), dtype=float32)
1 tf.Tensor(0.00020102292, shape=(), dtype=float32)
2 tf.Tensor(0.0001604848, shape=(), dtype=float32)
3 tf.Tensor(0.00013809305, shape=(), dtype=float32)
4 tf.Tensor(0.00013107198, shape=(), dtype=float32)
5 tf.Tensor(0.0001274135, shape=(), dtype=float32)
6 tf.Tensor(0.00011544105, shape=(), dtype=float32)
7 tf.Tensor(0.00010688149, shape=(), dtype=float32)
8 tf.Tensor(0.000102281156, shape=(), dtype=float32)
9 tf.Tensor(0.0001012246, shape=(), dtype=float32)
10 tf.Tensor(9.270781e-05, shape=(), dtype=float32)
11 tf.Tensor(8.65467e-05, shape=(), dtype=float32)
12 tf.Tensor(8.5385705e-05, shape=(), dtype=float32)
13 tf.Tensor(8.668405e-05, shape=(), dtype=float32)
14 tf.Tensor(8.456704e-05, shape=(), dtype=float32)
15 tf.Tensor(8.140916e-05, shape=(), dtype=float32)
16 tf.Tensor(8.1921506e-05, shape=(), dtype=float32)
17 tf.Tensor(8.126017e-05, shape=(), dtype=float32)
18 tf.Tensor(8.74001e-05, shape=(), dtype=float32)
19 tf.Tensor(8.

In [25]:
evaluate(model)

0.9779

## Baseline Model

In [8]:
baseline_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, 'softmax')
])

In [9]:
baseline_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [10]:
baseline_model.fit(
    x_train, y_train,
    epochs=20,
    validation_data=(x_test, y_test),
)

Epoch 1/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2590 - accuracy: 0.9273 - val_loss: 0.1392 - val_accuracy: 0.9569
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1139 - accuracy: 0.9659 - val_loss: 0.1087 - val_accuracy: 0.9676
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0792 - accuracy: 0.9753 - val_loss: 0.0807 - val_accuracy: 0.9755
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0596 - accuracy: 0.9812 - val_loss: 0.0784 - val_accuracy: 0.9752
Epoch 5/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0458 - accuracy: 0.9856 - val_loss: 0.0855 - val_accuracy: 0.9745
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0365 - accuracy: 0.9884 - val_loss: 0.0764 - val_accuracy: 0.9780
Epoch 7/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0279 - accuracy: 0.9914 - val_loss: 0.0791 - val_accuracy:

In [26]:
evaluate(baseline_model)

0.9771

## Model Robustness

Now, we compare the robustness of the model and the baseline. To do so, we add Gaussian noise to the test data and check the accuracy.

In [21]:
stddev = 2e-1
noise = tf.random.normal(tf.shape(x_test)) * stddev

In [34]:
def evaluate_robustness(model):
    accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test), axis=1))
    noised_accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test+noise), axis=1))
    print(f'Accuracy: {accuracy} -> {noised_accuracy}')

In [35]:
evaluate_robustness(model)

Accuracy: 0.9779 -> 0.9629


In [36]:
evaluate_robustness(baseline_model)

Accuracy: 0.9771 -> 0.9384


## Conclusion

By simply using the "gradient loss", we obtain a result that approaches the baseline. But the robustness is greatly out-performs the baseline.